In [1]:
#import modules
import os
import pickle
import pandas as pd
import numpy as np
from datetime import datetime
#See max columns in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
# Load the old dataset
# Read the CSV file with specified data types and skipped first 4 rows
old_df = pd.read_csv("data/southern_dallas_progress_export_20230906.csv")
old_df.head(2)


,Parcel ID,Adddress,Owner,Is Vacant Property,Property Type,Current Use,Year Built,Average Year Built By Council District,Owner Location,Owner Property Count,Public Owner,City or County Owned,Owner is Business,Owner Occupied,Suspected Heir Property,Bank Owned Property,Taxes Delinquent Since,Delinquent Taxes,Is Tax Delinquent,Code Liens - Count,Code Liens - Amount,Zip Code,Zoning,Council District,Vacant Since,Land Size,Unnamed: 26,Unnamed: 27
0,100561000000,"1109 WOOD ST, DALLAS, TX",BUNTING PARTNERS LP,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,1956.0,In City,20,NaN,No,Yes,No,No,No,NaN,NaN,No,NaN,NaN,75202,Central Area - CA-1(A)-Central Area,District 14,2022-04-01,0.124653,NaN,NaN
1,101971000000,"1908 ELM ST, DALLAS, TX",JARVIS BARNEY W JR TR & MARY LOU TR,Yes,Structure,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,1956.0,In City,1,NaN,No,No,Unknown,No,No,NaN,NaN,No,NaN,NaN,75201,Other - PD-619,District 14,2023-06-01,0.122321,NaN,NaN


In [3]:
# Load the new dataset

file_path = 'data/southern_dallas_progress_export_09102024.csv'
new_df = pd.read_csv(file_path)
new_df.head(2)

,Parcel ID,Address,Owner,Is Vacant Property,Property Type,Current Use,Year Built,Average Year Built By Council District,Owner Is Business,Owner Occupied,Suspected Heir Property,Taxes Delinquent Since,Delinquent Taxes,Is Tax Delinquent,Code Liens - Count,Code Liens - Amount,Zip Code,Zoning,Council Districts,Land Size,USPS Vacant Since,Owner Location,Owner Property Count,Owner Public Name,Owner Is Public,Is Bank Owned
0,00000100051000000,"400 MAIN ST, DALLAS, TX",NaN,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,1926.0,Unknown,Unknown,No,NaN,NaN,No,NaN,NaN,75202.0,Central Area - CA-1(A)-Central Area,District 2,0.918490,7/1/22,NaN,NaN,NaN,No,No
1,00000100561000000,"1109 WOOD ST, DALLAS, TX",BUNTING PARTNERS LP,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,1958.0,Yes,No,No,NaN,NaN,No,NaN,NaN,75202.0,Central Area - CA-1(A)-Central Area,District 14,0.124653,NaN,In MSA,20.0,NaN,No,No


In [4]:
# Use list comprehension to check that the columns are the same in both dataframes
print([col for col in new_df.columns if col not in old_df.columns])
print([col for col in old_df.columns if col not in new_df.columns])

['Address', 'Owner Is Business', 'Council Districts', 'USPS Vacant Since', 'Owner Public Name', 'Owner Is Public', 'Is Bank Owned']
['Adddress', 'Public Owner', 'City or County Owned', 'Owner is Business', 'Bank Owned Property', 'Council District', 'Vacant Since', 'Unnamed: 26', 'Unnamed: 27']


In [5]:
print(new_df.columns)

Index(['Parcel ID', 'Address', 'Owner', 'Is Vacant Property', 'Property Type',
       'Current Use', 'Year Built', 'Average Year Built By Council District',
       'Owner Is Business', 'Owner Occupied', 'Suspected Heir Property',
       'Taxes Delinquent Since', 'Delinquent Taxes', 'Is Tax Delinquent',
       'Code Liens - Count', 'Code Liens - Amount', 'Zip Code', 'Zoning',
       'Council Districts', 'Land Size', 'USPS Vacant Since', 'Owner Location',
       'Owner Property Count', 'Owner Public Name', 'Owner Is Public',
       'Is Bank Owned'],
      dtype='object')


In [6]:
# Value counts in the new_df 'Owner Public Name' field, including NaN
new_df['Owner Public Name'].value_counts(dropna=False)

NaN                            46328
City                            2744
Railway                          263
University                        83
County                            33
Transportation Department         28
City Community Development        25
Housing Authority                 24
College                           18
University Board of Regents       18
Other Schools                     16
US Postal Service                 12
US Government                     12
City Parks                         7
Turnpike Authority                 5
River Authority                    4
County Flood Control               4
City and County                    4
Charter School                     3
City Development                   3
City Real Estate                   1
FHLMC (Freddie Mac)                1
FNMA (Fannie Mae)                  1
City Finance                       1
Name: Owner Public Name, dtype: int64

In [7]:
# In new_df, add a new field called 'City or County Owned' that is False if 'Owner Public Name' is NaN, True otherwise
new_df['City or County Owned'] = new_df['Owner Public Name'].notnull()

Missing fields: 'Owner Location', 'Owner Property Count', 'Public Owner', 'City or County Owned', 'Bank Owned Property'

Requested new file export from Tolemi

In [8]:
# Missing fields: 'Owner Location', 'Owner Property Count', 'Public Owner','City or County Owned' , 'Bank Owned Property'


# In df, rename the following: 'Owner Is Business' to 'Owner is Business'; 'Council Districts' to 'Council District'; 

vacant_lots_df = new_df.rename(columns={'Owner Is Business': 'Owner is Business', 
                       'Council Districts': 'Council District', 
                       'Owner Is Public': 'Public Owner',
                       'Is Bank Owned': 'Bank Owned Property',
                       'USPS Vacant Since': 'Vacant Since'}
                       )
vacant_lots_df.head(2)

,Parcel ID,Address,Owner,Is Vacant Property,Property Type,Current Use,Year Built,Average Year Built By Council District,Owner is Business,Owner Occupied,Suspected Heir Property,Taxes Delinquent Since,Delinquent Taxes,Is Tax Delinquent,Code Liens - Count,Code Liens - Amount,Zip Code,Zoning,Council District,Land Size,Vacant Since,Owner Location,Owner Property Count,Owner Public Name,Public Owner,Bank Owned Property,City or County Owned
0,00000100051000000,"400 MAIN ST, DALLAS, TX",NaN,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,1926.0,Unknown,Unknown,No,NaN,NaN,No,NaN,NaN,75202.0,Central Area - CA-1(A)-Central Area,District 2,0.918490,7/1/22,NaN,NaN,NaN,No,No,False
1,00000100561000000,"1109 WOOD ST, DALLAS, TX",BUNTING PARTNERS LP,Yes,Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,1958.0,Yes,No,No,NaN,NaN,No,NaN,NaN,75202.0,Central Area - CA-1(A)-Central Area,District 14,0.124653,NaN,In MSA,20.0,NaN,No,No,False


In [9]:
# Use list comprehension to check that the columns are the same in both dataframes
print([col for col in vacant_lots_df.columns if col not in old_df.columns])
print([col for col in old_df.columns if col not in vacant_lots_df.columns])



['Address', 'Owner Public Name']
['Adddress', 'Unnamed: 26', 'Unnamed: 27']


In [10]:
#To see the rows before dropping duplicates
vacant_lots_df.shape

(49638, 27)

In [11]:
# Remove duplicates based on all columns
vacant_lots_df = vacant_lots_df.drop_duplicates(subset=['Parcel ID'])

In [12]:
# For vacant_lots_df['Council District'], extract the Council District Number. Example: 'District 1' -> 1
vacant_lots_df['Council District Number'] = vacant_lots_df['Council District'].str.extract('(\d+)')

# Convert Council District Number so that single digit districts have a leading zero
vacant_lots_df['Council District Number'] = vacant_lots_df['Council District Number'].str.zfill(2)

# Unique values in 'Council District Number'
vacant_lots_df['Council District Number'].unique()

# Set Council District to 'District' + 'Council District Number'
vacant_lots_df['Council District'] = 'District ' + vacant_lots_df['Council District Number']

# Value counts of 'Council District'
vacant_lots_df['Council District'].value_counts(sort=False)


District 02    4194
District 14    2183
District 01    2174
District 06    6633
District 07    6879
District 04    6032
District 09    1390
District 13    2136
District 11    1014
District 05    2543
District 08    5477
District 03    2539
District 10    1078
District 12     412
Name: Council District, dtype: int64

In [13]:
#Rows after duplicates have been dropped
vacant_lots_df.shape

(44715, 28)

In [14]:
# Recode 0 as NA in the "Year Built" column
vacant_lots_df['Year Built'] = np.where(vacant_lots_df['Year Built'] <= 222, np.nan, vacant_lots_df['Year Built'])
vacant_lots_df['Year Built'].value_counts()

2023.0    1850
2022.0    1773
2021.0    1232
1950.0     334
1955.0     273
1953.0     246
1952.0     240
1951.0     239
1954.0     225
1948.0     194
1940.0     187
1956.0     176
1949.0     174
1960.0     171
1959.0     160
1925.0     155
2005.0     154
1947.0     149
1958.0     144
1979.0     138
1961.0     136
1945.0     124
1962.0     120
1970.0     119
1946.0     116
1982.0     107
1957.0     107
1983.0     103
1930.0     102
1984.0      98
1964.0      92
1965.0      91
2020.0      90
1963.0      90
2004.0      89
1971.0      88
1985.0      85
1935.0      84
1967.0      83
2018.0      83
1972.0      82
1926.0      81
1980.0      79
1969.0      74
1920.0      73
1981.0      70
1941.0      68
1973.0      67
1978.0      66
2019.0      60
1968.0      60
1977.0      58
1974.0      58
2016.0      57
1976.0      54
1966.0      52
2003.0      50
1923.0      47
1924.0      47
2000.0      47
2017.0      47
1939.0      46
1999.0      45
1986.0      44
1938.0      44
1921.0      43
2006.0    

In [15]:
#Creates field Property Age Years in df

#Assign a variable to the current year
current_year = datetime.now().year
print(current_year)

# Calculate "Property Age Years" as the difference between current year and Year Built
vacant_lots_df['Property Age Years'] = current_year - vacant_lots_df['Year Built']

# Assign the calculated columns to variables for potential future use
property_age_years = vacant_lots_df['Property Age Years']


2024


In [16]:
#Creates field Vacancy Days in df

# Convert today's date to a Timestamp object
today = pd.Timestamp(datetime.today().date())

reference_date = pd.Timestamp('2024-09-10')
# reference_date = today

# Convert the 'Vacant Since' column to a Datetime column 
vacant_lots_df['Vacant Since'] = pd.to_datetime(vacant_lots_df['Vacant Since'])

# Calculate "Vacancy Days" as the difference between reference date and "Vacant Since" date
vacant_lots_df['Vacancy Days'] = (reference_date - vacant_lots_df['Vacant Since']).dt.days

# Assign the calculated columns to variables for potential future use
vacancy_days = vacant_lots_df['Vacancy Days']

vacancy_days.describe()

count    7319.000000
mean      623.188277
std       254.970762
min       132.000000
25%       372.000000
50%       802.000000
75%       802.000000
max       802.000000
Name: Vacancy Days, dtype: float64

In [17]:

#For ending set of 3 questions use a function to map Single Family,Multi, and Commercial
def zoning_triple_category(row):
    zoning=row['Zoning']
    
    if pd.notna(zoning):#check if thed value is not NaN
        if "Single Family" in zoning or "Mobile Home" in zoning:
            return "Single Family"
        elif any(keyword in zoning for keyword in ["Multifamily", "Duplex", "Cluster Housing", "Townhouse"]):
            return "Multifamily"
        # elif "Commercial" in zoning or "commercial" in zoning:
        elif any(keyword in zoning for keyword in ["Commercial", "Commerical", "commercial"]):
            return "Commercial"
        return "Other" #Return "Other" for NaN or missing values
vacant_lots_df['Zoning Category 3'] = vacant_lots_df.apply(zoning_triple_category, axis=1)  

In [18]:
# Check if "CHURCH" is a partial match in the "Current Use" column and create the "Owner is Church" column
vacant_lots_df['Owner is Church']=vacant_lots_df['Current Use'].str.contains('CHURCH', case=False, na=False)

# Map the True/False values to 'Yes' and 'No' in the "Owner is Church" column
vacant_lots_df['Owner is Church'] = vacant_lots_df['Owner is Church'].map({True: 'Yes', False: 'No'})

owner_is_church=vacant_lots_df["Owner is Church"]

In [19]:

#Checks strings in zoning column and maps them
def classify_zoning_category(row):
    zoning = row['Zoning']
    
    if pd.notna(zoning):  # Check if the value is not NaN
        if "Central Area" in zoning:
            return "Central Area"
        elif "Retail" in zoning or "Neighborhood Service" in zoning:
            return "Retail"
        elif any(keyword in zoning for keyword in ["Single Family", "Multifamily", "Duplex", "Cluster Housing", "Townhouse"]):
            return "Residential"
        elif "Office" in zoning:
            return "Office"
        elif "Industrial" in zoning:
            return "Industrial"
        #elif "Commercial" in zoning or "commercial" in zoning:  # Handle the typo
        elif any(keyword in zoning for keyword in ["Commercial", "Commerical", "commercial"]): # Handle the typo
            return "Commercial"
    return "Other"  # Return "Other" for NaN or missing values

# Apply the function to create the "Zoning Category" column
vacant_lots_df['Zoning Category'] = vacant_lots_df.apply(classify_zoning_category, axis=1)

In [20]:
# Define a function to apply the classification logic
def classify_owner(row):
    if pd.notna(row['Public Owner']):
        return row['Public Owner']
    elif row['Public Owner'] == 'City of Dallas':
        return 'City of Dallas'
    elif row['Public Owner'] == 'City of Dallas Housing Authority':
        return 'City of Dallas Housing Authority'
    elif row['Public Owner'] == 'County of Dallas':
        return 'County of Dallas'  
    elif row['Owner is Church'] == 'Yes':
        return 'Church'
    elif row['Owner is Business'] == 'Yes':
        return 'Business'
    elif row['Bank Owned Property'] == 'Yes':
        return 'Bank'
    else:
        return 'Private Owner'

# Apply the function to create the "Owner" column
vacant_lots_df['Owner'] = vacant_lots_df.apply(classify_owner, axis=1)
   

In [21]:
# Selecting columns of interest and filtering by 'Property Type' == 'Structure'
structure_df = vacant_lots_df[vacant_lots_df['Property Type'] == 'Structure']

# Selecting columns of interest and filtering by 'Property Type' == 'Land' for land dataframe
land_df = vacant_lots_df[vacant_lots_df['Property Type'] == 'Land']

# Filter residential properties
residential_properties = vacant_lots_df[vacant_lots_df['Zoning Category'] == 'Residential']

# Filter commercial properties used Zone Category 3 column for zoned single family residential, commercial and multifamily questions
commercial_properties = vacant_lots_df[vacant_lots_df['Zoning Category 3'] == 'Commercial']


In [22]:
vacant_lots_df.groupby('Property Type')['Property Type'].count()

Property Type
Land         28300
Structure    16415
Name: Property Type, dtype: int64

In [23]:
vacant_lots_df.shape

(44715, 33)

In [24]:
vacant_lots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44715 entries, 0 to 49637
Data columns (total 33 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Parcel ID                               44715 non-null  object        
 1   Address                                 44715 non-null  object        
 2   Owner                                   44715 non-null  object        
 3   Is Vacant Property                      44715 non-null  object        
 4   Property Type                           44715 non-null  object        
 5   Current Use                             44715 non-null  object        
 6   Year Built                              12710 non-null  float64       
 7   Average Year Built By Council District  44684 non-null  float64       
 8   Owner is Business                       44715 non-null  object        
 9   Owner Occupied                          44715 non-

In [25]:
vacant_lots_df.isnull().agg(['sum'])

,Parcel ID,Address,Owner,Is Vacant Property,Property Type,Current Use,Year Built,Average Year Built By Council District,Owner is Business,Owner Occupied,Suspected Heir Property,Taxes Delinquent Since,Delinquent Taxes,Is Tax Delinquent,Code Liens - Count,Code Liens - Amount,Zip Code,Zoning,Council District,Land Size,Vacant Since,Owner Location,Owner Property Count,Owner Public Name,Public Owner,Bank Owned Property,City or County Owned,Council District Number,Property Age Years,Vacancy Days,Zoning Category 3,Owner is Church,Zoning Category
sum,0,0,0,0,0,0,32005,31,0,0,0,39339,39339,0,38019,38019,4,8638,31,0,37396,3791,3791,41517,0,0,0,31,32005,37396,8638,0,0


In [26]:
vacant_lots_df.columns

Index(['Parcel ID', 'Address', 'Owner', 'Is Vacant Property', 'Property Type',
       'Current Use', 'Year Built', 'Average Year Built By Council District',
       'Owner is Business', 'Owner Occupied', 'Suspected Heir Property',
       'Taxes Delinquent Since', 'Delinquent Taxes', 'Is Tax Delinquent',
       'Code Liens - Count', 'Code Liens - Amount', 'Zip Code', 'Zoning',
       'Council District', 'Land Size', 'Vacant Since', 'Owner Location',
       'Owner Property Count', 'Owner Public Name', 'Public Owner',
       'Bank Owned Property', 'City or County Owned',
       'Council District Number', 'Property Age Years', 'Vacancy Days',
       'Zoning Category 3', 'Owner is Church', 'Zoning Category'],
      dtype='object')

# Check vacancy dates

In [27]:
vacant_lots_df['Land Size'].describe()

count    44715.000000
mean         1.340455
std         27.316516
min          0.000000
25%          0.134100
50%          0.180391
75%          0.369136
max       5322.378832
Name: Land Size, dtype: float64

In [28]:
pd.unique(vacant_lots_df['Vacant Since'])

array(['2022-07-01T00:00:00.000000000',                           'NaT',
       '2023-09-04T00:00:00.000000000', '2023-03-01T00:00:00.000000000',
       '2024-03-04T00:00:00.000000000', '2023-12-01T00:00:00.000000000',
       '2024-05-01T00:00:00.000000000', '2022-09-01T00:00:00.000000000',
       '2022-12-02T00:00:00.000000000', '2023-05-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [29]:
vacant_lots_df['Vacant Since'].value_counts(dropna=False, sort=False).sort_index()

2022-07-01     4519
2022-09-01      154
2022-12-02      204
2023-03-01      319
2023-05-01       15
2023-09-04      534
2023-12-01      357
2024-03-04      763
2024-05-01      454
NaT           37396
Name: Vacant Since, dtype: int64

# Analyses

In [30]:
#Sets results for dict
results = {}

In [31]:
#1.How many vacant lots are in each council district?

#Groups Council distrcits column and 
vacant_lots_counts_by_district  = land_df.groupby('Council District').size()


In [32]:
# Create a helper function that will convert a series of counts to a pandas dataframe, with the counts column renamed as 'count'
def counts_to_dataframe(counts_series, column_name='count'):
    # Detect name of the index and use it as the column name
    index_name = counts_series.index.name
    if index_name is None:
        index_name = 'index'
    # Reset the index to convert it to a DataFrame
    df = counts_series.reset_index()
    # Rename the columns
    df.columns = [index_name, column_name]
    # Set index_name column as the index
    df.set_index(index_name, inplace=True)
   
    return df

In [33]:
# Convert vacant_lots_counts_by_district to a pandas dataframe
vacant_lots_counts_by_district_df = counts_to_dataframe(vacant_lots_counts_by_district)
vacant_lots_counts_by_district_df

,count
Council District,
District 01,1415
District 02,2480
District 03,1909
District 04,3838
District 05,1800
District 06,4647
District 07,4962
District 08,3896
District 09,310


In [34]:
#dictonary that starts the result values
results["How many vacant lots are in each council district?"] = {
    'question_number': 1,
    'value': vacant_lots_counts_by_district_df
}

results["How many vacant lots are in each council district?"]


{'question_number': 1,
 'value':                   count
 Council District       
 District 01        1415
 District 02        2480
 District 03        1909
 District 04        3838
 District 05        1800
 District 06        4647
 District 07        4962
 District 08        3896
 District 09         310
 District 10         540
 District 11         469
 District 12         244
 District 13         625
 District 14        1135}

In [35]:
#2 How long have the vacant lots in each council district been vacant? 
vacant_lots_stats = land_df.groupby('Council District')['Vacancy Days'].agg(['count', 'min','median', 'mean', 'max'])

# Rename the columns for clarity
vacant_lots_stats.columns = ['Count', 'Min Days', 'Median Days', 'Mean Days', 'Max Days']
vacant_lots_stats

,Count,Min Days,Median Days,Mean Days,Max Days
Council District,,,,,
District 01,42,372.0,802.0,771.285714,802.0
District 02,76,132.0,802.0,719.868421,802.0
District 03,31,802.0,802.0,802.000000,802.0
District 04,211,132.0,802.0,750.995261,802.0
District 05,65,284.0,802.0,779.953846,802.0
District 06,155,132.0,802.0,784.612903,802.0
District 07,257,132.0,802.0,704.743191,802.0
District 08,86,132.0,802.0,693.476744,802.0
District 09,13,190.0,802.0,675.230769,802.0


In [36]:

results["How long have the vacant lots in each council district been vacant?"] = {
    'question_number': 2,
    'value': vacant_lots_stats
}


results["How long have the vacant lots in each council district been vacant?"]

{'question_number': 2,
 'value':                   Count  Min Days  Median Days   Mean Days  Max Days
 Council District                                                    
 District 01          42     372.0        802.0  771.285714     802.0
 District 02          76     132.0        802.0  719.868421     802.0
 District 03          31     802.0        802.0  802.000000     802.0
 District 04         211     132.0        802.0  750.995261     802.0
 District 05          65     284.0        802.0  779.953846     802.0
 District 06         155     132.0        802.0  784.612903     802.0
 District 07         257     132.0        802.0  704.743191     802.0
 District 08          86     132.0        802.0  693.476744     802.0
 District 09          13     190.0        802.0  675.230769     802.0
 District 10          18     190.0        802.0  696.611111     802.0
 District 11          15     190.0        802.0  732.533333     802.0
 District 12           1     802.0        802.0  802.00000

In [37]:
# 3 How many are zoned single family residential, commercial and multifamily? <-- rows are council District; columns are counts within each of the 3 zoning types

# Group by 'Council District' and 'Zoning Category', then count occurrences
#zoning_category_counts = vacant_lots_df.groupby(['Council District', 'Zoning Category']).size().unstack(fill_value=0)


In [38]:

'''results[" How many are zoned single family residential, commercial and multifamily? <-- rows are council District; columns are counts within each of the 3 zoning types"] = {
    'question_number': 3,
    'value': zoning_category_counts
}

results'''

'results[" How many are zoned single family residential, commercial and multifamily? <-- rows are council District; columns are counts within each of the 3 zoning types"] = {\n    \'question_number\': 3,\n    \'value\': zoning_category_counts\n}\n\nresults'

In [39]:
#3 How many vacant lots in each council district are zoned single family residential, commercial and multifamily 

# Group by 'Council District' and 'Zoning Category', then count occurrences
zoning_category_counts_3 = land_df.groupby(['Council District', 'Zoning Category 3']).size().unstack(fill_value=0)


In [40]:

results["How many are zoned single family residential, commercial and multifamily?"] = {
    'question_number': 3,
    'value': zoning_category_counts_3
}


results["How many are zoned single family residential, commercial and multifamily?"]

{'question_number': 3,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Council District                                                
 District 01                 3           76    995            284
 District 02               208          281   1465            127
 District 03                59          124    932            766
 District 04               108          302   1494           1928
 District 05                16           49    802            933
 District 06               128           92   2113           1429
 District 07               164          277    980            427
 District 08               245          115   1155           2368
 District 09                12           16    115             53
 District 10                10           95    296             55
 District 11                 2           26    330            111
 District 12                 0           50    105             89
 District 13                 3           91 

In [41]:
# 4. How many vacant structures are in each council district?

# Groups Council District by structure and vacant property

# Grouping by 'Council District' and counting occurrences
vacant_structure_count = structure_df.groupby('Council District').size()

# Convert vacant_structure_count to a pandas dataframe
vacant_structure_count_df = counts_to_dataframe(vacant_structure_count)

In [42]:

results["How many vacant structures are in each council district?"] = {
    'question_number': 4,
    'value': vacant_structure_count_df
}

results["How many vacant structures are in each council district?"]

{'question_number': 4,
 'value':                   count
 Council District       
 District 01         759
 District 02        1714
 District 03         630
 District 04        2194
 District 05         743
 District 06        1986
 District 07        1917
 District 08        1581
 District 09        1080
 District 10         538
 District 11         545
 District 12         168
 District 13        1511
 District 14        1048}

In [43]:
#5 How long have the vacant structures in each council district been vacant? (see question 2)
vacant_structure_count_days = structure_df.groupby('Council District')['Vacancy Days'].agg(['count', 'min','median', 'mean', 'max'])

# Rename the columns for clarity
vacant_structure_count_days.columns = ['Count', 'Min Days', 'Median Days', 'Mean Days', 'Max Days']


In [44]:

results["How long have the vacant structures in each council district been vacant? (see question 2)"] = {
    'question_number': 5,
    'value': vacant_structure_count_days
}

results["How long have the vacant structures in each council district been vacant? (see question 2)"]

{'question_number': 5,
 'value':                   Count  Min Days  Median Days   Mean Days  Max Days
 Council District                                                    
 District 01         315     132.0        802.0  693.733333     802.0
 District 02         709     132.0        559.0  561.737659     802.0
 District 03         241     132.0        802.0  622.178423     802.0
 District 04        1141     132.0        802.0  677.190184     802.0
 District 05         367     132.0        802.0  680.896458     802.0
 District 06         683     132.0        802.0  662.519766     802.0
 District 07         691     132.0        802.0  595.374819     802.0
 District 08         386     132.0        802.0  625.880829     802.0
 District 09         475     132.0        559.0  503.275789     802.0
 District 10         202     132.0        372.0  454.618812     802.0
 District 11         163     132.0        740.0  533.723926     802.0
 District 12          48     132.0        559.0  566.81250

In [45]:

#6 How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)

zoning_category_counts_structure = structure_df.groupby(['Council District', 'Zoning Category 3']).size().unstack(fill_value=0)


In [46]:

results["How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)"] = {
    'question_number': 6,
    'value': zoning_category_counts_structure
}

results["How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)"] 


{'question_number': 6,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Council District                                                
 District 01                 4           41    396            283
 District 02                53          334    607             82
 District 03                 6           23    164            433
 District 04                14          100    320           1758
 District 05                 9           37    103            594
 District 06                16           94    716            845
 District 07                32          149    300            290
 District 08                25           30    406           1119
 District 09                 7           58    170             76
 District 10                 8           36    222            102
 District 11                 0           36    256            253
 District 12                 3           26     77             62
 District 13                 0           61 

In [47]:
#7.How many vacant lots are in each zip code?

# Groups the zip code and if property is vacant and produces the count 
vacant_lots_counts_by_zip = land_df.groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_lots_counts_by_zip.index = vacant_lots_counts_by_zip.index.astype(int)

In [48]:
# Convert vacant_lots_counts_by_zip to a pandas dataframe
vacant_lots_counts_by_zip_df = counts_to_dataframe(vacant_lots_counts_by_zip)

In [49]:

results["How many vacant lots are in each zip code?"] = {
    'question_number': 7,
    'value': vacant_lots_counts_by_zip_df
}

results["How many vacant lots are in each zip code?"]

{'question_number': 7,
 'value':           count
 Zip Code       
 75006         7
 75019        66
 75041         1
 75043        11
 75050         1
 75051        18
 75060         2
 75062         3
 75063        11
 75080        14
 75081         2
 75104         4
 75134         1
 75141         8
 75149         9
 75150        14
 75159        10
 75181         4
 75182         8
 75201       362
 75202       154
 75203      2285
 75204       523
 75205        42
 75206       245
 75207       473
 75208       966
 75209       172
 75210      1050
 75211       956
 75212      2685
 75214       195
 75215      2935
 75216      2122
 75217      2084
 75218       117
 75219       351
 75220       457
 75223       844
 75224       282
 75225       114
 75226       479
 75227       626
 75228       283
 75229       376
 75230       249
 75231       224
 75232       350
 75233       194
 75234        25
 75235       365
 75236       601
 75237       324
 75238       136
 75240       114

In [50]:
#8 How long have the vacant lots in each zip code been vacant?
vacant_zips_count_days = land_df.groupby('Zip Code')['Vacancy Days'].agg(['count', 'min','median', 'mean', 'max'])

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_zips_count_days.index = vacant_zips_count_days.index.astype(int)

# Rename the columns for clarity
vacant_zips_count_days.columns = ['Count', 'Min Days', 'Median Days', 'Mean Days', 'Max Days']

In [51]:

results["How long have the vacant lots in each zip code been vacant?"] = {
    'question_number': 8,
    'value': vacant_zips_count_days
}

results["How long have the vacant lots in each zip code been vacant?"] 

{'question_number': 8,
 'value':           Count  Min Days  Median Days   Mean Days  Max Days
 Zip Code                                                    
 75006         0       NaN          NaN         NaN       NaN
 75019         0       NaN          NaN         NaN       NaN
 75041         0       NaN          NaN         NaN       NaN
 75043         0       NaN          NaN         NaN       NaN
 75050         0       NaN          NaN         NaN       NaN
 75051         0       NaN          NaN         NaN       NaN
 75060         0       NaN          NaN         NaN       NaN
 75062         0       NaN          NaN         NaN       NaN
 75063         0       NaN          NaN         NaN       NaN
 75080         0       NaN          NaN         NaN       NaN
 75081         0       NaN          NaN         NaN       NaN
 75104         0       NaN          NaN         NaN       NaN
 75134         0       NaN          NaN         NaN       NaN
 75141         0       NaN          Na

In [52]:
#9 How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?


zoning_category_counts_zip = land_df.groupby(['Zip Code', 'Zoning Category 3']).size().unstack(fill_value=0)

# Convert index (zip codes) to integers to remove the .0 decimal part
zoning_category_counts_zip.index = zoning_category_counts_zip.index.astype(int)

In [53]:

results["How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?"] = {
    'question_number': 9,
    'value': zoning_category_counts_zip
}

results["How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?"]

{'question_number': 9,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Zip Code                                                        
 75006                       0            0      6              0
 75019                       0            0     12              0
 75043                       0            0      7              0
 75051                       1            0     11              6
 75060                       0            0      1              0
 75062                       0            0      3              0
 75063                       0            0      2              0
 75080                       0            0     14              0
 75081                       0            1      1              0
 75104                       0            0      3              1
 75134                       0            0      0              1
 75141                       0            0      8              0
 75149                       0            0 

In [54]:
#10 How many vacant structures are in each zip code?

# Groups the zip code and if property is vacant and produces the count 
vacant_structure_counts_by_zip = structure_df.groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_structure_counts_by_zip.index = vacant_structure_counts_by_zip.index.astype(int)

# Convert vacant_structure_counts_by_zip to a pandas dataframe
vacant_structure_counts_by_zip_df = counts_to_dataframe(vacant_structure_counts_by_zip)

In [55]:

results["How many vacant structures are in each zip code?"] = {
    'question_number': 10,
    'value': vacant_structure_counts_by_zip_df
}

results["How many vacant structures are in each zip code?"]

{'question_number': 10,
 'value':           count
 Zip Code       
 75006         1
 75019         1
 75042         2
 75043         1
 75051         6
 75062         1
 75063         1
 75080        36
 75081         1
 75115         1
 75150         1
 75201       171
 75202        42
 75203       519
 75204       363
 75205        98
 75206       388
 75207       208
 75208       396
 75209       422
 75210       353
 75211       421
 75212       961
 75214       399
 75215       821
 75216      1722
 75217       825
 75218       527
 75219       300
 75220       451
 75223       304
 75224       210
 75225       229
 75226        90
 75227       508
 75228      1055
 75229       624
 75230       386
 75231       282
 75232       287
 75233       114
 75234        24
 75235       273
 75236        82
 75237        70
 75238       170
 75240       196
 75241       700
 75243       312
 75244        95
 75246        42
 75247        93
 75248       204
 75249        67
 75251        1

In [56]:
#11 How long have the vacant structures in each zip code been vacant? (see question 8)
vacant_zips_structure_count_days = structure_df.groupby('Zip Code')['Vacancy Days'].agg(['count', 'min','median', 'mean', 'max'])

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_zips_structure_count_days.index = vacant_zips_structure_count_days.index.astype(int)

# Rename the columns for clarity
vacant_zips_structure_count_days.columns = ['Count', 'Min Days', 'Median Days', 'Mean Days', 'Max Days']

In [57]:

results["How long have the vacant structures in each zip code been vacant? (see question 8)"] = {
    'question_number': 11,
    'value': vacant_zips_structure_count_days
}

results["How long have the vacant structures in each zip code been vacant? (see question 8)"]

{'question_number': 11,
 'value':           Count  Min Days  Median Days   Mean Days  Max Days
 Zip Code                                                    
 75006         0       NaN          NaN         NaN       NaN
 75019         0       NaN          NaN         NaN       NaN
 75042         2     190.0        419.0  419.000000     648.0
 75043         0       NaN          NaN         NaN       NaN
 75051         0       NaN          NaN         NaN       NaN
 75062         0       NaN          NaN         NaN       NaN
 75063         0       NaN          NaN         NaN       NaN
 75080         0       NaN          NaN         NaN       NaN
 75081         0       NaN          NaN         NaN       NaN
 75115         0       NaN          NaN         NaN       NaN
 75150         0       NaN          NaN         NaN       NaN
 75201        48     372.0        740.0  651.687500     802.0
 75202         6     372.0        802.0  730.333333     802.0
 75203       213     132.0        802

In [58]:
#12 How many vacant structures in each zip code are zoned single family residential, commercial and multifamily? (see question 9)

zoning_category_counts_structure_zip = structure_df.groupby(['Zip Code', 'Zoning Category 3']).size().unstack(fill_value=0)

# Convert index (zip codes) to integers to remove the .0 decimal part
zoning_category_counts_structure_zip.index = zoning_category_counts_structure_zip.index.astype(int)

In [59]:

results["How many vacant structures in each zip code are zoned single family residential, commercial and multifamily? (see question 9)"] = {
    'question_number': 12,
    'value': zoning_category_counts_structure_zip
}

results["How many vacant structures in each zip code are zoned single family residential, commercial and multifamily? (see question 9)"]


{'question_number': 12,
 'value': Zoning Category 3  Commercial  Multifamily  Other  Single Family
 Zip Code                                                        
 75006                       0            0      1              0
 75042                       0            0      1              1
 75051                       0            0      5              1
 75062                       0            0      1              0
 75080                       0            0     35              1
 75081                       0            0      1              0
 75115                       0            0      1              0
 75150                       0            0      1              0
 75201                       0            0    118              0
 75202                       0            0     42              0
 75203                       1           34    270            213
 75204                       4          196    132              3
 75205                       0            1

In [60]:
#13 What is the total amount of back taxes owed on vacant land in each City Council District?

# Grouping and calculating the total amount of back taxes owed on vacant land by City Council District
#Remove vacant prop yes filter
back_taxes_total_by_district = land_df.groupby('Council District')['Delinquent Taxes'].sum()

# Convert back_taxes_total_by_district to a pandas dataframe
back_taxes_total_by_district_df = counts_to_dataframe(back_taxes_total_by_district, column_name='sum')

In [61]:

results["What is the total amount of back taxes owed on vacant land in each City Council District?"] = {
    'question_number': 13,
    'value': back_taxes_total_by_district_df
}

results["What is the total amount of back taxes owed on vacant land in each City Council District?"] 

{'question_number': 13,
 'value':                        sum
 Council District          
 District 01        6160.27
 District 02       11819.58
 District 03        4750.14
 District 04       18380.90
 District 05        4320.63
 District 06       24181.61
 District 07       14308.83
 District 08       20124.42
 District 09         323.05
 District 10        4755.90
 District 11         248.13
 District 12          43.13
 District 13        1304.02
 District 14        3511.00}

In [62]:
#14 What is the total amount of back taxes owed on vacant land in each zip code?

# Grouping and calculating the total amount of back taxes owed on vacant land by Zip Code
back_taxes_total_by_zip = land_df.groupby('Zip Code')['Delinquent Taxes'].sum()

# Convert index (zip codes) to integers to remove the .0 decimal part
back_taxes_total_by_zip.index = back_taxes_total_by_zip.index.astype(int)

# Convert back_taxes_total_by_zip to a pandas dataframe
back_taxes_total_by_zip_df = counts_to_dataframe(back_taxes_total_by_zip, column_name='sum')


In [63]:

results["What is the total amount of back taxes owed on vacant land in each zip code?"] = {
    'question_number': 14,
    'value': back_taxes_total_by_zip_df
}

results["What is the total amount of back taxes owed on vacant land in each zip code?"] 

{'question_number': 14,
 'value':                sum
 Zip Code          
 75006         0.00
 75019         0.45
 75041         0.00
 75043         0.00
 75050         0.00
 75051         5.22
 75060         0.00
 75062         0.00
 75063         0.06
 75080         0.02
 75081         0.18
 75104         0.00
 75134         0.00
 75141         0.00
 75149         0.00
 75150         0.04
 75159         0.00
 75181         0.00
 75182       203.39
 75201      1402.29
 75202       634.58
 75203     10415.79
 75204      3792.79
 75205        14.26
 75206        68.61
 75207       284.97
 75208      4725.41
 75209       599.87
 75210      3201.71
 75211      4278.49
 75212     16279.24
 75214       152.60
 75215      8310.52
 75216      9869.90
 75217      5985.56
 75218         0.01
 75219      2589.59
 75220      2615.65
 75223      3204.18
 75224       968.65
 75225        50.89
 75226      2206.49
 75227      2261.92
 75228       192.07
 75229       281.48
 75230         2.47
 75231 

In [64]:
#15 What is the total amount of back taxes owed on vacant structures in each City Council District?

# Grouping and calculating the total amount of back taxes owed on vacant land by City Council District
back_taxes_total_by_district_by_structure = structure_df.groupby('Council District')['Delinquent Taxes'].sum()

# Convert back_taxes_total_by_district to a pandas dataframe
back_taxes_total_by_district_by_structure_df = counts_to_dataframe(back_taxes_total_by_district_by_structure, column_name='sum')


In [65]:

results["What is the total amount of back taxes owed on vacant structures in each City Council District?"] = {
    'question_number': 15,
    'value': back_taxes_total_by_district_by_structure_df
}

results["What is the total amount of back taxes owed on vacant structures in each City Council District?"]

{'question_number': 15,
 'value':                        sum
 Council District          
 District 01        3296.95
 District 02       41146.59
 District 03        4803.20
 District 04       18218.75
 District 05        2558.69
 District 06       23400.64
 District 07       10657.86
 District 08        6932.20
 District 09       10969.02
 District 10        8667.99
 District 11        3462.20
 District 12        1906.72
 District 13       20408.81
 District 14       18883.96}

In [66]:
#16 What is the total amount of back taxes owed on vacant structures in each zip code?

# Grouping and calculating the total amount of back taxes owed on vacant land by Zip Code
back_taxes_total_by_zip_by_structure = structure_df.groupby('Zip Code')['Delinquent Taxes'].sum()

# Convert index (zip codes) to integers to remove the .0 decimal part
back_taxes_total_by_zip_by_structure.index = back_taxes_total_by_zip_by_structure.index.astype(int)

# Convert back_taxes_total_by_zip to a pandas dataframe
back_taxes_total_by_zip_by_structure_df = counts_to_dataframe(back_taxes_total_by_zip_by_structure, column_name='sum')


In [67]:

results["What is the total amount of back taxes owed on vacant structures in each zip code?"] = {
    'question_number': 16,
    'value': back_taxes_total_by_zip_by_structure_df
}

results["What is the total amount of back taxes owed on vacant structures in each zip code?"] 

{'question_number': 16,
 'value':                sum
 Zip Code          
 75006         0.00
 75019         0.00
 75042         0.00
 75043         0.00
 75051         0.00
 75062         0.00
 75063         0.00
 75080         0.00
 75081         0.00
 75115         0.00
 75150         0.01
 75201      8971.84
 75202      3600.70
 75203      2813.61
 75204      7380.81
 75205         0.00
 75206      3970.48
 75207      4181.55
 75208      3471.32
 75209      8779.79
 75210      1292.39
 75211      2820.36
 75212      8142.31
 75214      9185.06
 75215      7023.30
 75216     14260.40
 75217      3824.90
 75218      6334.18
 75219      3844.48
 75220      5347.09
 75223      3088.73
 75224      1005.65
 75225        73.51
 75226        18.43
 75227      3055.09
 75228      5870.56
 75229      6810.31
 75230      3160.24
 75231      5775.98
 75232      1188.48
 75233       749.48
 75234         8.22
 75235     16778.79
 75236      1127.63
 75237      1003.16
 75238       556.57
 75240 

In [68]:
#17 What is the average age of residential properties in each council district?

#avg_age__prop_District=vacant_lots_df.groupby('Council District')['Property Age Years'].mean()
# Calculate the average age of residential properties

residential_avg_age_prop_District = residential_properties.groupby('Council District')['Property Age Years'].mean()

# Convert residential_avg_age_prop_District to a pandas dataframe
residential_avg_age_prop_District_df = counts_to_dataframe(residential_avg_age_prop_District, column_name='mean')

In [69]:

results["What is the average age of residential properties in each council district?"] = {
    'question_number': 17,
    'value': residential_avg_age_prop_District_df
}

results["What is the average age of residential properties in each council district?"]

{'question_number': 17,
 'value':                        mean
 Council District           
 District 01       64.513109
 District 02       25.467797
 District 03       42.898667
 District 04       50.881302
 District 05       46.398833
 District 06       20.174691
 District 07       35.020888
 District 08       27.926554
 District 09       53.660194
 District 10       46.239669
 District 11       38.524590
 District 12       42.860759
 District 13       26.819005
 District 14       17.762712}

In [70]:
#18 What is the average age of residential properties in each zip code?
avg_age__prop_zip = residential_properties.groupby('Zip Code')['Property Age Years'].agg(['count', 'min','median', 'mean', 'max'])

#Fills na values with 0
#avg_age__prop_zip=avg_age__prop_zip.fillna(0)

#converts avg age to int data type to remve decimals
avg_age__prop_zip.index=avg_age__prop_zip.index.astype(int)


In [71]:

results["What is the average age of residential properties in each zip code?"] = {
    'question_number': 18,
    'value': avg_age__prop_zip
}

results["What is the average age of residential properties in each zip code?"]

{'question_number': 18,
 'value':           count   min  median       mean    max
 Zip Code                                       
 75042         1  46.0    46.0  46.000000   46.0
 75051         0   NaN     NaN        NaN    NaN
 75080         0   NaN     NaN        NaN    NaN
 75081         0   NaN     NaN        NaN    NaN
 75104         0   NaN     NaN        NaN    NaN
 75134         0   NaN     NaN        NaN    NaN
 75159         0   NaN     NaN        NaN    NaN
 75203       205   1.0    64.0  45.175610  104.0
 75204       129   1.0     2.0  11.031008  112.0
 75205         0   NaN     NaN        NaN    NaN
 75206       125   1.0     3.0  14.352000  104.0
 75208        74   1.0    77.5  54.486486  111.0
 75209       109   1.0     3.0  21.779817   85.0
 75210        18   2.0    59.5  64.333333  113.0
 75211       235   1.0    72.0  55.527660  114.0
 75212       517   1.0     2.0  15.943907   94.0
 75214        50   1.0    18.0  36.360000  111.0
 75215         0   NaN     NaN      

In [72]:
#19 What is the average size of residential properties in each council district?


# Group by council district and calculate the average property size
average_size_by_district = residential_properties.groupby('Council District')['Land Size'].mean()

# Convert average_size_by_district to a pandas dataframe
average_size_by_district_df = counts_to_dataframe(average_size_by_district, column_name='mean')

In [73]:

results["What is the average size of residential properties in each council district?"] = {
    'question_number': 19,
    'value': average_size_by_district_df
}

results["What is the average size of residential properties in each council district?"]

{'question_number': 19,
 'value':                       mean
 Council District          
 District 01       0.685420
 District 02       0.357161
 District 03       1.583366
 District 04       0.347238
 District 05       0.866688
 District 06       0.240354
 District 07       1.493583
 District 08       1.634822
 District 09       0.658111
 District 10       1.198724
 District 11       1.483045
 District 12       1.713730
 District 13       1.024812
 District 14       0.315840}

In [74]:
#20 What is the average size of residential properties in each zip code?

# Group by council district and calculate the average property size
average_size_by_zip = residential_properties.groupby('Zip Code')['Land Size'].mean()

# Convert index (zip codes) to integers to remove the .0 decimal part
average_size_by_zip.index = average_size_by_zip.index.astype(int)

# Convert average_size_by_zip to a pandas dataframe
average_size_by_zip_df = counts_to_dataframe(average_size_by_zip, column_name='mean')


In [75]:

results["What is the average size of residential properties in each zip code?"] = {
    'question_number': 20,
    'value': average_size_by_zip_df
}

results["What is the average size of residential properties in each zip code?"]

{'question_number': 20,
 'value':                mean
 Zip Code           
 75042      0.192886
 75051      8.861988
 75080     21.995701
 75081      0.107777
 75104      0.272161
 75134      0.164271
 75159      0.230563
 75203      0.229449
 75204      0.200162
 75205      0.155813
 75206      0.297336
 75208      0.679906
 75209      0.707452
 75210      1.206632
 75211      1.041779
 75212      0.219995
 75214      0.447470
 75215     41.161512
 75216      0.387215
 75217      1.295802
 75218      0.820805
 75220      0.453314
 75223      0.277663
 75224      0.609624
 75225      1.092759
 75226      0.204128
 75227      1.089873
 75228      1.433934
 75229      0.785560
 75230      0.718089
 75231      4.595392
 75232      0.892594
 75233      0.888135
 75234      0.198871
 75235      0.343190
 75236      3.003959
 75237      1.935544
 75238      0.281128
 75240      1.173141
 75241      1.426822
 75243      1.328064
 75244      0.396240
 75246      0.290583
 75248      1.489770
 

In [76]:
#21 What is the average age of commercial properties in each council district?

# Group by council district and calculate the average property size by commercial prop
average_age_by_district_commercial = commercial_properties.groupby('Council District')['Property Age Years'].mean()

# Convert average_age_by_district_commercial to a pandas dataframe
average_age_by_district_commercial_df = counts_to_dataframe(average_age_by_district_commercial, column_name='mean')


In [77]:

results["What is the average age of commercial properties in each council district?"] = {
    'question_number': 21,
    'value': average_age_by_district_commercial_df
}

results["What is the average age of commercial properties in each council district?"]

{'question_number': 21,
 'value':                        mean
 Council District           
 District 01       70.500000
 District 02       62.187500
 District 03       27.000000
 District 04       70.333333
 District 05       41.250000
 District 06       55.166667
 District 07       67.761905
 District 08        1.000000
 District 09       34.200000
 District 10       38.333333
 District 11             NaN
 District 12             NaN
 District 13             NaN
 District 14       10.000000}

In [78]:
#22 What is the average age of commercial properties in each zip code?

average_age_by_zip_commercial = commercial_properties.groupby('Zip Code')['Property Age Years'].agg(['count', 'min','median', 'mean', 'max'])

# Convert index (zip codes) to integers to remove the .0 decimal part
average_age_by_zip_commercial.index = average_age_by_zip_commercial.index.astype(int)

In [79]:
results["What is the average age of commercial properties in each zip code?"] = {
    'question_number': 22,
    'value': average_age_by_zip_commercial
}

results["What is the average age of commercial properties in each zip code?"]

{'question_number': 22,
 'value':           count   min  median       mean    max
 Zip Code                                       
 75051         0   NaN     NaN        NaN    NaN
 75203         1  89.0    89.0  89.000000   89.0
 75204         4  73.0    77.5  76.750000   79.0
 75206         1  10.0    10.0  10.000000   10.0
 75207         0   NaN     NaN        NaN    NaN
 75208         4  49.0    66.5  63.500000   72.0
 75209         0   NaN     NaN        NaN    NaN
 75210         0   NaN     NaN        NaN    NaN
 75211         1  27.0    27.0  27.000000   27.0
 75212         2  49.0    62.5  62.500000   76.0
 75215         0   NaN     NaN        NaN    NaN
 75216        18   3.0    69.0  64.444444   99.0
 75217         4  13.0    47.5  41.250000   57.0
 75218         2  52.0    59.0  59.000000   66.0
 75220         1  29.0    29.0  29.000000   29.0
 75223        23   4.0    73.0  65.652174  104.0
 75224         0   NaN     NaN        NaN    NaN
 75226         2  44.0    51.5  51.5

In [80]:
#23 What is the total amount of city liens owed on vacant land in each City Council District?

# Grouping and calculating the total amount of city liens owed on vacant land by City Council District
city_liens_total_by_district = land_df.groupby('Council District')['Code Liens - Amount'].sum()

# Convert city_liens_total_by_district to a pandas dataframe
city_liens_total_by_district_df = counts_to_dataframe(city_liens_total_by_district, column_name='sum')


In [81]:

results["What is the total amount of city liens owed on vacant land in each City Council District?"] = {
    'question_number': 23,
    'value': city_liens_total_by_district_df
}

results["What is the total amount of city liens owed on vacant land in each City Council District?"]

{'question_number': 23,
 'value':                          sum
 Council District            
 District 01        141560.34
 District 02        301761.51
 District 03        214713.02
 District 04       2596592.50
 District 05        568502.05
 District 06        598888.25
 District 07       6154314.74
 District 08        917787.65
 District 09         10875.65
 District 10          8425.69
 District 11          9983.31
 District 12             0.00
 District 13          3592.27
 District 14         28926.43}

In [82]:
#24 What is the total amount of city liens owed on vacant land in each zip code?

# Grouping and calculating the total amount of city liens owed on vacant land by City Council District
city_liens_total_by_zip = land_df.groupby('Zip Code')['Code Liens - Amount'].sum()

# Convert index (zip codes) to integers to remove the .0 decimal part
city_liens_total_by_zip.index = city_liens_total_by_zip.index.astype(int)

# Convert city_liens_total_by_zip to a pandas dataframe
city_liens_total_by_zip_df = counts_to_dataframe(city_liens_total_by_zip, column_name='sum')


In [83]:

results["What is the total amount of city liens owed on vacant land in each zip code?"] = {
    'question_number': 24,
    'value': city_liens_total_by_zip_df
}

results["What is the total amount of city liens owed on vacant land in each zip code?"]

{'question_number': 24,
 'value':                  sum
 Zip Code            
 75006           0.00
 75019           0.00
 75041           0.00
 75043           0.00
 75050           0.00
 75051        3603.91
 75060           0.00
 75062           0.00
 75063           0.00
 75080           0.00
 75081           0.00
 75104           0.00
 75134           0.00
 75141           0.00
 75149         171.41
 75150           0.00
 75159           0.00
 75181           0.00
 75182           0.00
 75201           0.00
 75202           0.00
 75203     1048815.36
 75204       67764.97
 75205           0.00
 75206        6015.15
 75207        5301.83
 75208       51626.08
 75209        6726.82
 75210     1908422.49
 75211      105539.91
 75212      518121.36
 75214        9092.80
 75215     3688922.43
 75216     1749897.67
 75217      602389.17
 75218        4457.29
 75219       11869.36
 75220       36165.00
 75223      399820.30
 75224       79375.76
 75225         239.61
 75226       75991.57

In [84]:
#25 What is the total amount of city liens owed on vacant structures in each City Council District?

# Grouping and calculating the total amount of city liens owed on vacant structures by City Council District
city_liens_total_by_district_by_structure = structure_df.groupby('Council District')['Code Liens - Amount'].sum()

# Convert city_liens_total_by_district to a pandas dataframe
city_liens_total_by_district_by_structure_df = counts_to_dataframe(city_liens_total_by_district_by_structure, column_name='sum')


In [85]:

results["What is the total amount of city liens owed on vacant structures in each City Council District?"] = {
    'question_number': 25,
    'value': city_liens_total_by_district_by_structure_df
}

results["What is the total amount of city liens owed on vacant structures in each City Council District?"]

{'question_number': 25,
 'value':                          sum
 Council District            
 District 01         60956.36
 District 02         96414.16
 District 03         96242.48
 District 04       1182617.26
 District 05        222558.07
 District 06        217224.27
 District 07       1056353.08
 District 08        308517.40
 District 09         70502.58
 District 10          4796.08
 District 11         34294.92
 District 12         49869.82
 District 13         12647.82
 District 14         18767.31}

In [86]:
#26 How many properties are in the name of heirship in the City of Dallas?

#Use suspected heir prop col and combine with public owner filter
# Filter the DataFrame for properties in the City of Dallas with 'Suspected Heir Property' == 'Yes'
#Would get zero combined with City of Dallas filter
dallas_heirship_properties = (vacant_lots_df['Suspected Heir Property'] == 'Yes').sum()

# Convert single value dallas_heirship_properties to a pandas dataframe
dallas_heirship_properties_df = pd.DataFrame({'count': [dallas_heirship_properties]}, index=[0])
dallas_heirship_properties_df



,count
0,292


In [87]:

results["How many properties are in the name of heirship in the City of Dallas?"] = {
    'question_number': 26,
    'value': dallas_heirship_properties
}

results["How many properties are in the name of heirship in the City of Dallas?"] 

{'question_number': 26, 'value': 292}

In [88]:
#27 How many properties in the name of heirship in the City of Dallas are vacant in each zip code?

#apply Suspected heir to Yes and group by zip code
dallas_heirship_zip = vacant_lots_df[vacant_lots_df['Suspected Heir Property'] == 'Yes'].groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
dallas_heirship_zip.index = dallas_heirship_zip.index.astype(int)

# Convert dallas_heirship_zip to a pandas dataframe
dallas_heirship_zip_df = counts_to_dataframe(dallas_heirship_zip)


In [89]:

results["How many properties in the name of heirship in the City of Dallas are vacant in each zip code?"] = {
    'question_number': 27,
    'value': dallas_heirship_zip_df
}

results["How many properties in the name of heirship in the City of Dallas are vacant in each zip code?"] 

{'question_number': 27,
 'value':           count
 Zip Code       
 75201         4
 75203        24
 75206         2
 75207         1
 75208         6
 75209         1
 75210        16
 75211         6
 75212        28
 75214         4
 75215        35
 75216        55
 75217         9
 75218         6
 75220         1
 75223         9
 75224         4
 75225         4
 75226         5
 75227         7
 75228        13
 75229         2
 75230         3
 75231         2
 75232         4
 75233         3
 75235         1
 75237         2
 75238         1
 75241        19
 75243         4
 75246         1
 75249         2
 75253         8}

In [90]:
#28 How many properties in the name of heirship in the City of Dallas are vacant in each council district?

#apply Suspected heir to Yes and group by council distrcit
dallas_heirship_District = vacant_lots_df[vacant_lots_df['Suspected Heir Property'] == 'Yes'].groupby('Council District').size()

# Convert dallas_heirship_District to a pandas dataframe
dallas_heirship_District_df = counts_to_dataframe(dallas_heirship_District)


In [91]:

results["How many properties in the name of heirship in the City of Dallas are vacant in each council district?"] = {
    'question_number': 28,
    'value': dallas_heirship_District_df
}

results["How many properties in the name of heirship in the City of Dallas are vacant in each council district?"] 

{'question_number': 28,
 'value':                   count
 Council District       
 District 01           9
 District 02          18
 District 03          13
 District 04          65
 District 05          13
 District 06          33
 District 07          75
 District 08          29
 District 09          12
 District 10           7
 District 11           2
 District 13           7
 District 14           9}

# Analysis of private owners with 5 or more vacant properties

In [92]:
file_owners5plus = os.path.join('data', 'data-3_own_5_or_more_properties_2024-09-11.csv')
owners5plus_df = pd.read_csv(file_owners5plus, skiprows=3)
owners5plus_df.head()

,Parcel ID,Address,Owns At Least Five Vacant Properties,Property Type,Is Vacant Property,Zip Code,Council Districts,Owner_1,OwnAddr_1,Owner_2,OwnAddr_2,Owner_3,OwnAddr_3
0,"=""00000104485000000""","912 ROSS AVE, DALLAS, TX",Yes,Land,Yes,75202,District 14,1001 SAN JACINTO LLC,"4265 SAN FELIPE ST STE 550, HOUSTON, TEXAS 770...",NaN,NaN,NaN,NaN
1,"=""00000104509000000""","911 SAN JACINTO ST, DALLAS, TX",Yes,Land,Yes,75202,District 14,1001 SAN JACINTO LLC,"4265 SAN FELIPE ST STE 550, HOUSTON, TEXAS 770...",NaN,NaN,NaN,NaN
2,"=""00000104533000000""","909 SAN JACINTO ST, DALLAS, TX",Yes,Land,Yes,75202,District 14,1001 SAN JACINTO LLC,"4265 SAN FELIPE ST STE 550, HOUSTON, TEXAS 770...",NaN,NaN,NaN,NaN
3,"=""00000104515000000""","1001 SAN JACINTO ST, DALLAS, TX",Yes,Land,Yes,75202,District 14,1001 SAN JACINTO LLC,"4265 SAN FELIPE ST STE 550, HOUSTON, TEXAS 770...",NaN,NaN,NaN,NaN
4,"=""00000104503000000""","915 SAN JACINTO ST, DALLAS, TX",Yes,Land,Yes,75202,District 14,1001 SAN JACINTO LLC,"4265 SAN FELIPE ST STE 550, HOUSTON, TEXAS 770...",NaN,NaN,NaN,NaN


In [93]:
# Rename 'Council Districts' to 'Council District' in owners5plus_df
owners5plus_df.rename(columns={'Council Districts': 'Council District'}, inplace=True)

# For owners5plus_df['Council District'], extract the Council District Number. Example: 'District 1' -> 1
owners5plus_df['Council District Number'] = owners5plus_df['Council District'].str.extract('(\d+)')

# Convert Council District Number so that single digit districts have a leading zero
owners5plus_df['Council District Number'] = owners5plus_df['Council District Number'].str.zfill(2)

# Unique values in 'Council District Number'
print(owners5plus_df['Council District Number'].unique())

# Set Council District to 'District' + 'Council District Number'
owners5plus_df['Council District'] = 'District ' + owners5plus_df['Council District Number']

# Value counts of 'Council District'
owners5plus_df['Council District'].value_counts(sort=False)

['14' '06' '02' '03' '01' '07' '08' '13' '04' '05' '12' '09' '11' '10' nan]


District 14     281
District 06    1437
District 02     664
District 03     566
District 01     360
District 07    1221
District 08     436
District 13     247
District 04     776
District 05     151
District 12      83
District 09      17
District 11     246
District 10      93
Name: Council District, dtype: int64

In [94]:
# Count the number of properties owned by each owner
owner_vacant_property_counts = owners5plus_df['Owner_1'].value_counts()

# Convert owner_property_counts to a pandas dataframe
owner_vacant_property_counts_df = counts_to_dataframe(owner_vacant_property_counts)

# Filter for owners with 5 or more properties
owner_vacant_property_counts_df = owner_vacant_property_counts_df[owner_vacant_property_counts_df['count'] >= 5]

owner_vacant_property_counts_df.head()

,count
index,
AR & PR LLC,135
TEXAS SFI PARTNERSHIP 63 LTD,96
DALLAS HOUSING ACQUISITION & DEVELOPMENT CORP,95
Contreras Antonio,84
SIERRA LINDA LIMITED LIABILITY,68


In [95]:
# Join the owner_vacant_property_counts_df with the owners5plus_df on 'Owner_1'
owners5plus_df_merged =  owners5plus_df.merge(owner_vacant_property_counts_df, left_on='Owner_1', right_index=True, suffixes=('', '_count'))

# Filter for owners with 5 or more properties
owners5plus_df_merged = owners5plus_df_merged[owners5plus_df_merged['count'] >= 5]

# Rename "Council Districts" to "Council District"
#owners5plus_df_merged.rename(columns={'Council Districts': 'Council District'}, inplace=True)

owners5plus_df_merged.head(10)

,Parcel ID,Address,Owns At Least Five Vacant Properties,Property Type,Is Vacant Property,Zip Code,Council District,Owner_1,OwnAddr_1,Owner_2,OwnAddr_2,Owner_3,OwnAddr_3,Council District Number,count
0,"=""00000104485000000""","912 ROSS AVE, DALLAS, TX",Yes,Land,Yes,75202,District 14,1001 SAN JACINTO LLC,"4265 SAN FELIPE ST STE 550, HOUSTON, TEXAS 770...",NaN,NaN,NaN,NaN,14,6
1,"=""00000104509000000""","911 SAN JACINTO ST, DALLAS, TX",Yes,Land,Yes,75202,District 14,1001 SAN JACINTO LLC,"4265 SAN FELIPE ST STE 550, HOUSTON, TEXAS 770...",NaN,NaN,NaN,NaN,14,6
2,"=""00000104533000000""","909 SAN JACINTO ST, DALLAS, TX",Yes,Land,Yes,75202,District 14,1001 SAN JACINTO LLC,"4265 SAN FELIPE ST STE 550, HOUSTON, TEXAS 770...",NaN,NaN,NaN,NaN,14,6
3,"=""00000104515000000""","1001 SAN JACINTO ST, DALLAS, TX",Yes,Land,Yes,75202,District 14,1001 SAN JACINTO LLC,"4265 SAN FELIPE ST STE 550, HOUSTON, TEXAS 770...",NaN,NaN,NaN,NaN,14,6
4,"=""00000104503000000""","915 SAN JACINTO ST, DALLAS, TX",Yes,Land,Yes,75202,District 14,1001 SAN JACINTO LLC,"4265 SAN FELIPE ST STE 550, HOUSTON, TEXAS 770...",NaN,NaN,NaN,NaN,14,6
5,"=""00000104539000000""","913 SAN JACINTO ST, DALLAS, TX",Yes,Land,Yes,75202,District 14,1001 SAN JACINTO LLC,"4265 SAN FELIPE ST STE 550, HOUSTON, TEXAS 770...",NaN,NaN,NaN,NaN,14,6
6,"=""00000601357000000""","10630 GOODNIGHT LN, DALLAS, TX",Yes,Land,Yes,75220,District 06,10562 GL LLC,"45 MAIN ST STE 502, BROOKLYN, NEW YORK 112011093",NaN,NaN,NaN,NaN,06,10
7,"=""00000601426000000""","10603 GOODNIGHT LN, DALLAS, TX",Yes,Land,Yes,75220,District 06,10562 GL LLC,"45 MAIN ST STE 502, BROOKLYN, NEW YORK 112011093",NaN,NaN,NaN,NaN,06,10
8,"=""00000601441000000""","10663 GOODNIGHT LN, DALLAS, TX",Yes,Land,Yes,75220,District 06,10562 GL LLC,"45 MAIN ST STE 502, BROOKLYN, NEW YORK 112011093",NaN,NaN,NaN,NaN,06,10
9,"=""00000601438000000""","10649 GOODNIGHT LN, DALLAS, TX",Yes,Land,Yes,75220,District 06,10562 GL LLC,"45 MAIN ST STE 502, BROOKLYN, NEW YORK 112011093",NaN,NaN,NaN,NaN,06,10


In [96]:
#29 How many private owners have five or more vacant properties in the City of Dallas?
count_owners = owner_vacant_property_counts_df.shape[0]

In [97]:

results["How many private owners have five or more vacant properties in the City of Dallas?"] = {
    'question_number': 29,
    'value': count_owners
}

results["How many private owners have five or more vacant properties in the City of Dallas?"] 

{'question_number': 29, 'value': 607}

In [98]:
# By zip code, count the number of owners (Owner_1) in owners5plus_df_merged
owner_count_by_zip = owners5plus_df_merged.groupby('Zip Code')['Owner_1'].nunique()

# Convert owner_count_by_zip to a pandas dataframe
owner_count_by_zip_df = counts_to_dataframe(owner_count_by_zip)

# Rename count field to 'Owner Count'
owner_count_by_zip_df.rename(columns={'count': 'Owner Count'}, inplace=True)
owner_count_by_zip_df.head(10)

,Owner Count
Zip Code,
75019,3
75050,1
75051,1
75080,1
75081,1
75159,1
75201,20
75202,10
75203,92


In [99]:
# By zip code, count the number of parcels (Parcel ID) in owners5plus_df_merged
parcel_count_by_zip = owners5plus_df_merged.groupby('Zip Code')['Parcel ID'].nunique()

# Convert parcel_count_by_zip to a pandas dataframe
parcel_count_by_zip_df = counts_to_dataframe(parcel_count_by_zip)

# Rename count field to 'Parcel Count'
parcel_count_by_zip_df.rename(columns={'count': 'Parcel Count'}, inplace=True)
parcel_count_by_zip_df.head(10)

,Parcel Count
Zip Code,
75019,13
75050,1
75051,2
75080,14
75081,1
75159,2
75201,128
75202,50
75203,622


In [100]:
#30 How many private owners with five or more vacant properties in the City of Dallas are in each zip code?

# Merge owner_count_by_zip_df and parcel_count_by_zip_df on the index (Zip Code)
owner_parcel_count_by_zip_df = owner_count_by_zip_df.merge(parcel_count_by_zip_df, left_index=True, right_index=True)
owner_parcel_count_by_zip_df.head(10)


,Owner Count,Parcel Count
Zip Code,,
75019,3,13
75050,1,1
75051,1,2
75080,1,14
75081,1,1
75159,1,2
75201,20,128
75202,10,50
75203,92,622


In [101]:

results["How many private owners with five or more vacant properties in the City of Dallas are in each zip code?"] = {
    'question_number': 30,
    'value': owner_parcel_count_by_zip_df
}

results["How many private owners with five or more vacant properties in the City of Dallas are in each zip code?"]

{'question_number': 30,
 'value':           Owner Count  Parcel Count
 Zip Code                           
 75019               3            13
 75050               1             1
 75051               1             2
 75080               1            14
 75081               1             1
 75159               1             2
 75201              20           128
 75202              10            50
 75203              92           622
 75204              23            90
 75205               4            20
 75206               9            42
 75207              19           113
 75208              38           237
 75209               5            26
 75210              53           287
 75211              39           298
 75212             101           911
 75214               4             7
 75215             120           670
 75216              82           350
 75217              40           174
 75218               3             7
 75219              19           141
 7522

In [102]:
# By council district, count the number of owners (Owner_1) in owners5plus_df_merged
owner_count_by_district = owners5plus_df_merged.groupby('Council District')['Owner_1'].nunique()

# Convert owner_count_by_district to a pandas dataframe
owner_count_by_district_df = counts_to_dataframe(owner_count_by_district)

# Rename count field to 'Owner Count'
owner_count_by_district_df.rename(columns={'count': 'Owner Count'}, inplace=True)
owner_count_by_district_df.head(10)

,Owner Count
Council District,
District 01,52
District 02,110
District 03,63
District 04,129
District 05,38
District 06,156
District 07,160
District 08,96
District 09,6


In [103]:
# By council district, count the number of parcels (Parcel ID) in owners5plus_df_merged
parcel_count_by_district = owners5plus_df_merged.groupby('Council District')['Parcel ID'].nunique()

# Convert parcel_count_by_district to a pandas dataframe
parcel_count_by_district_df = counts_to_dataframe(parcel_count_by_district)

# Rename count field to 'Parcel Count'
parcel_count_by_district_df.rename(columns={'count': 'Parcel Count'}, inplace=True)

parcel_count_by_district_df.head(10)

,Parcel Count
Council District,
District 01,360
District 02,663
District 03,564
District 04,775
District 05,150
District 06,1403
District 07,1206
District 08,424
District 09,13


In [104]:
#31 How many private owners with five or more vacant properties in the City of Dallas are in each council district?

# Merge owner_count_by_zip_df and parcel_count_by_zip_df on the index (Zip Code)
owner_parcel_count_by_district_df = owner_count_by_district_df.merge(parcel_count_by_district_df, left_index=True, right_index=True)
owner_parcel_count_by_district_df

,Owner Count,Parcel Count
Council District,,
District 01,52,360
District 02,110,663
District 03,63,564
District 04,129,775
District 05,38,150
District 06,156,1403
District 07,160,1206
District 08,96,424
District 09,6,13


In [105]:

results["How many private owners with five or more vacant properties in the City of Dallas are in each council district?"] = {
    'question_number': 31,
    'value': owner_parcel_count_by_district_df
}

results["How many private owners with five or more vacant properties in the City of Dallas are in each council district?"]

{'question_number': 31,
 'value':                   Owner Count  Parcel Count
 Council District                           
 District 01                52           360
 District 02               110           663
 District 03                63           564
 District 04               129           775
 District 05                38           150
 District 06               156          1403
 District 07               160          1206
 District 08                96           424
 District 09                 6            13
 District 10                15            92
 District 11                17           243
 District 12                 9            77
 District 13                26           247
 District 14                47           273}

In [106]:
#32 How many churches own vacant properties in the City of Dallas?

church_owners = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes']

num_church_properties = church_owners.shape[0]



In [107]:
# Using church_owners, count number of properties grouped by property type
church_property_types = church_owners.groupby('Property Type').size()

# Convert to dict
church_property_types = church_property_types.to_dict()

# Insert total number of church-owned properties
church_property_types['Total'] = num_church_properties

# Convert to a dataframe
church_property_types = pd.DataFrame(church_property_types, index=[0])

church_property_types

,Land,Structure,Total
0,2,49,51


In [108]:

results["How many churches own vacant properties in the City of Dallas?"] = {
    'question_number': 32,
    'value': church_property_types
}

results["How many churches own vacant properties in the City of Dallas?"] 

{'question_number': 32,
 'value':    Land  Structure  Total
 0     2         49     51}

In [109]:
#33 How many churches owned vacant properties in the City of Dallas are in each zip code?

church_prop_zip = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes'].groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
church_prop_zip.index = church_prop_zip.index.astype(int)

# Convert church_prop_zip to a pandas dataframe
church_prop_zip_df = counts_to_dataframe(church_prop_zip)


In [110]:

results["How many churches owned vacant properties in the City of Dallas are in each zip code?"] = {
    'question_number': 33,
    'value': church_prop_zip_df
}

results["How many churches owned vacant properties in the City of Dallas are in each zip code?"] 

{'question_number': 33,
 'value':           count
 Zip Code       
 75203         2
 75211         2
 75212         5
 75215         4
 75216         7
 75217         4
 75223         1
 75224         2
 75226         2
 75227         3
 75228         1
 75232         2
 75233         2
 75237         1
 75238         1
 75241        10
 75243         1
 75254         1}

In [111]:
#34 How many are in each council district

church_prop_district = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes'].groupby('Council District').size()

# Convert church_prop_district to a pandas dataframe
church_prop_district_df = counts_to_dataframe(church_prop_district)


In [112]:

results["How many are in each council district"] = {
    'question_number': 34,
    'value': church_prop_district_df
}

results["How many are in each council district"]

{'question_number': 34,
 'value':                   count
 Council District       
 District 01           2
 District 02           3
 District 03           4
 District 04           9
 District 05           3
 District 06           5
 District 07          10
 District 08          12
 District 09           1
 District 10           1
 District 11           1}

# Save result to disk as Pickle object

In [113]:
pkl_filename = "vacant_lots_analysis_results.pkl"
pickle.dump( results, open( pkl_filename, "wb" ) )

In [114]:
# Sample code for loading it back into memory
# results = pickle.load( open( pkl_filename, "rb" ) )

In [115]:
import os

# Use os to create a folder called 'excel-exports' to store the Excel files
export_folder = 'excel-exports'
os.makedirs(export_folder, exist_ok=True)


In [116]:
# Export results dataframes to Excel, with each question as a separate sheet named using the qusetion number. Example "Q-1"
# The key should appear in the 0,0 position of the sheet
# The data should appear in the 3,2 position of the sheet
# RUN THIS CELL ONLY ONCE
export_file = os.path.join(export_folder, 'southern_dallas_progress_export_09102024_results_001.xlsx')
with pd.ExcelWriter(export_file) as writer:
    for key, value in results.items():
        try:
            # If value is a single value (for example, numpy.int64 or int), convert it to a DataFrame
            if isinstance(value['value'], (np.int64, int)):
                df = pd.DataFrame({'count': value['value']}, index=[0])
                df.to_excel(writer, sheet_name=f'Q-{value["question_number"]}', startrow=2, startcol=3, index=True)
            else:
                value['value'].to_excel(writer, sheet_name=f'Q-{value["question_number"]}', startrow=2, startcol=3, index=True)
                
            # Write the key (question label) to the first row of the sheet
            pd.DataFrame([key]).to_excel(writer, sheet_name=f'Q-{value["question_number"]}', index=False, header=False)
            
            # Auto-adjust columns width
            #for column in value['value'].columns:
            #    writer.sheets[f'Q-{value["question_number"]}'].set_column(column, column, max(len(column), value['value'][column].astype(str).str.len().max()) + 2)
        except Exception as e:
            n = value['question_number']
            print(f'Error exporting Q-{n} {key}: {e}')
            continue